In [2]:
# -*- coding: UTF-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from tqdm import tqdm
from scipy.fftpack import fft
from matplotlib.pylab import mpl
import csv
import array
import sqlite3
import pprint
from matplotlib.ticker import FuncFormatter
from matplotlib import ticker, cm
import matplotlib
from collections import Counter
from mpl_toolkits.mplot3d import Axes3D
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.utils import check_random_state
from sklearn.decomposition import PCA, FastICA
from sklearn.cluster import KMeans
from scipy.optimize import curve_fit
from scipy.interpolate import make_interp_spline
from scipy.signal import savgol_filter
import warnings
import pywt


%matplotlib qt
warnings.filterwarnings("ignore")
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
mpl.rcParams['axes.unicode_minus'] = False  #显示负号

### Extract PAC values

In [ ]:
#  ID DDD HH:MM:SS.mmmuuun      PARA1  CH  RISE  COUN  ENER  DURATION AMP  ABS-ENERGY
path = r'E:\data\pac\316L-1.5-annealed-AE-3 sensor-20201228\316L-1.5-annealed-AE-3 sensor-20201228_2.TXT'
with open(path, 'r') as f:
    data = np.array([np.array(i.strip("\n").split()) for i in f.readlines()[8:1305403]])

In [ ]:
valid_idx = np.where((data[:, 4] == '1') & (data[:, 6].astype('int') > 3) & (data[:, -2].astype('int') != 24))[0] # & (data[:, -2].astype('int') != 24)
valid_data = data[valid_idx]
valid_data.shape

In [ ]:
abnormal_1 = np.array([116, 81, 26, 44, 8, 70, 54, 75, 13, 35, 2, 37, 129, 64])
abnormal_2 = np.array([59, 88, 48, 17, 62, 72, 119, 16, 21, 64, 9, 1])
abnormal_3 = np.array([88, 22, 107, 62, 101, 72, 119, 21, 64, 1, 138, 98])

In [ ]:
np.where((Dur > 31.2) & (Dur < 31.8) & (Eny > 2) & (Eny < 5))[0], Dur[np.where((Dur > 31.2) & (Dur < 31.8) & (Eny > 2) & (Eny < 5))[0]]

In [ ]:
Time[abnormal_3], Amp[abnormal_3], Dur[abnormal_3], Eny[abnormal_3]

In [ ]:
time = np.round(all_Time,4)
valid_idx_origin, valid_idx_cal = [], []
for idx, t in enumerate(np.round(valid_time,4)):
    res = np.where(time == t)[0]
    if res.shape[0] > 1:
        cha = np.array([abs(time[res[0]]-t), abs(time[res[1]]-t)])
        tmp = np.argmin(cha)
        valid_idx_origin.append(res[tmp])
        valid_idx_cal.append(idx)
    elif res.shape[0] == 1:
        valid_idx_origin.append(res[0])
        valid_idx_cal.append(idx)

In [ ]:
valid_data = data[valid_idx_origin]
valid_data.shape

In [ ]:
all_Time = []
for j in data[:, 2]:
    t = 0
    for n, i in zip([2, 1, 0], j.split(':')):
        t += float(i)*pow(60, n)
    all_Time.append(t)
all_Time = np.array(all_Time)

In [ ]:
Time = []
for j in valid_data[:, 2]:
    t = 0
    for n, i in zip([2, 1, 0], j.split(':')):
        t += float(i)*pow(60, n)
    Time.append(t)
Time = np.array(Time)

In [ ]:
df_1 = pd.DataFrame({'Time':Time, 'Amplitude':Amp, 'RiseTime':RiseT, 'Duration':Dur, 'Energy':Eny, 'ABS_ENERGY':abs_eny, 'Counts':Counts})
df_1.to_csv('316L_PAC.csv')

In [ ]:
Amp = valid_data[:, -2].astype('float')
RiseT = valid_data[:, 5].astype('float')
Dur = 20 * np.log10(valid_data[:, 8].astype('float'))
Eny = 20 * np.log10(valid_data[:, -1].astype('float'))
eny = valid_data[:, -4].astype('float')
Counts = valid_data[:, 6].astype('int')
abs_eny = valid_data[:, -1].astype('float')

In [ ]:
np.where((Time > 882) & (Time <= 885))[0]

In [ ]:
valid_eny = [3, 2127, 16, 1128, 483]
Time[[770, 771, 772]]

In [ ]:
fig = plt.figure(figsize=[6, 3.9])
ax = plt.subplot()
# ax.set_yscale("log", nonposy='clip')
# ax.set_xscale("log", nonposx='clip')
ax.scatter(Dur, Eny, s=15, c=color_2)
ax.scatter(Dur[abnormal_2], Eny[abnormal_2], s=15, c='black')
# ax.scatter(Amp[valid_eny], Eny[valid_eny], s=15, c=color_1)
plot_norm(ax, '20log(Dur)', '20log(Abs-Eny)', legend=False)
# '20log(Amp)', '20log(Abs-Eny)', 'Duration (μs)'

### Plot format

In [7]:
color_1 = [255/255, 0/255, 102/255] # red
color_2 = [0/255, 136/255, 204/255] # blue
xlabelz = ['Amplitude (μV)', 'Duration (μs)', 'Energy (aJ)']
def plot_norm(ax, xlabel=None, ylabel=None, zlabel=None, title=None, x_lim=[], y_lim=[], z_lim=[], legend=True, grid=False, 
              legend_loc='upper left', font_color='black', legendsize=11, labelsize=14, titlesize=15, ticksize=13, linewidth=2):
    ax.spines['bottom'].set_linewidth(linewidth)
    ax.spines['left'].set_linewidth(linewidth)
    ax.spines['right'].set_linewidth(linewidth)
    ax.spines['top'].set_linewidth(linewidth)

    # 设置坐标刻度值的大小以及刻度值的字体 Arial
    ax.tick_params(which='both', width=linewidth, labelsize=ticksize, colors=font_color)
    labels = ax.get_xticklabels() + ax.get_yticklabels()
    [label.set_fontname('Arial') for label in labels]

    font_legend = {'family': 'Arial', 'weight': 'normal', 'size': legendsize}
    font_label = {'family': 'Arial', 'weight': 'bold', 'size': labelsize, 'color':font_color}
    font_title = {'family': 'Arial', 'weight': 'bold', 'size': titlesize, 'color':font_color}

    if x_lim:
        ax.set_xlim(x_lim[0], x_lim[1])
    if y_lim:
        ax.set_ylim(y_lim[0], y_lim[1])
    if z_lim:
        ax.set_zlim(z_lim[0], z_lim[1])
    if legend:
        plt.legend(loc=legend_loc, prop=font_legend)
    if grid:
        ax.grid(ls='-.')
    if xlabel:
        ax.set_xlabel(xlabel, font_label)
    if ylabel:
        ax.set_ylabel(ylabel, font_label)
    if zlabel:
        ax.set_zlabel(zlabel, font_label)
    if title:
        ax.set_title(title, font_title)
    plt.tight_layout()

In [51]:
def cal_wave(name, status, magnification_dB=40, thr_dB=25):
    magnification = pow(10, magnification_dB / 20)
    thr_μV = pow(10, thr_dB / 20)
    with open(name, "r") as f:
        for _ in range(3):
            f.readline()
        sample_interval = float(f.readline()[29:])
        for _ in range(3):
            f.readline()
        points_num = int(f.readline()[36:])
        channel_num = int(f.readline().strip()[16:])
        hit_num = int(f.readline()[12:])
        t0 = float(f.readline()[14:])
        sig = np.array([float(i.strip("\n")) for i in f.readlines()[1:]]) / magnification * pow(10, 6)
#         time_label = np.linspace(t0, t0 + sample_interval * (points_num - 1), points_num)
        time = np.linspace(0, sample_interval * (points_num - 1) * pow(10, 6), points_num)
#         if valid:
        valid_wave_idx = np.where(abs(sig) >= thr_μV)[0]
        start = time[valid_wave_idx[0]]
        end = time[valid_wave_idx[-1]]
        duration = end - start
        sig = sig[valid_wave_idx[0]:(valid_wave_idx[-1] + 1)]
        time = np.linspace(0, duration, sig.shape[0])
    
    fig = plt.figure(figsize=(6, 4.1), num='Waveform--Time {} (s) & Duration {} (μs)'.format(t0, duration))
    fig.text(0.95, 0.17, status, fontdict={'family': 'Arial', 'fontweight': 'bold', 'fontsize': 12},
             horizontalalignment="right")
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(time, sig, lw=1)
    plt.axhline(abs(thr_μV), 0, sig.shape[0], linewidth=1, color="black")
    plt.axhline(-abs(thr_μV), 0, sig.shape[0], linewidth=1, color="black")
    plot_norm(ax, 'Time (μs)', 'Amplitude (μV)', title='Time: %f (s)' % t0, legend=False, grid=True)
    return time, sig

In [48]:
path = r'E:\data\pac\316L-1.5-annealed-AE-3 sensor-20201228'
os.chdir(path)

In [52]:
for name in os.listdir(path)[2:-1]:
    time, sig = cal_wave(name, '316L')
# 20 * np.log10(max(abs(sig))), 20 * np.log10(496)

In [54]:
20 * np.log10(300)

49.54242509439325